<a href="https://colab.research.google.com/github/ppseguel/jobskillmatching/blob/master/industry_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import dataset
#follow this documentation, using Github, https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

import pandas as pd

url = 'https://raw.githubusercontent.com/ppseguel/jobskillmatching/master/marketingintern_industry.csv'
job_data = pd.read_csv(url)
#job_data = pd.read_csv(url, sep=',').values
#job_data = pd.DataFrame(job_data)
#job_data = job_data.values

#filling blank cells
df = pd.DataFrame(job_data)
df1 = df.fillna(0)

df1.columns
df1 = df1.rename(columns={"Unnamed: 0": "id", "Linkedin/bloomberg":"LinkedInBloomerg"})

# Dataset is now stored in a Pandas Dataframe

#dataset = p.read_csv('dataset.csv', sep=',').values
#dataset = p.DataFrame(dataset)
#dataset = dataset.values


#from sklearn.preprocessing import Imputer
#imputer = Imputer(missing_values='NaN', strategy='mean', axis = 0)
#imputer = imputer.fit(job_data[:, 2:6])
#job_data[:, 2:6] = imputer.transform(job_data[:, 2:6])
#dataset = p.DataFrame(dataset)
#dataset = dataset.values


In [0]:
#turning textual data to numerical
#from sklearn.preprocessing import LabelEncoder
#labelencoder_0 = LabelEncoder() #independent variable encoder
#job_data[:,0] = labelencoder_0.fit_transform(job_data[:,0])
#labelencoder_1 = LabelEncoder() #independent variable encoder
#dataset[:,1] = labelencoder_1.fit_transform(job_data[:,1])
#labelencoder_6 = LabelEncoder() #dependent (target) variable encoder
#job_data[:,6] = labelencoder_6.fit_transform(job_data[:,6])
#dataset = p.DataFrame(dataset)
#dataset = dataset.values


In [0]:
#Checking the dataset
#print(job_data)
#print(df1)
#df1.head()
#job_data.head()

#print(job_data.Groups, job_data.tableau)


In [0]:
#EXPERIMENT 1 (EDIT DATASET)

# 1. Construct Datasets for Training and Evaluation
#selecting y and x variables

#splitting the dataset into the source variables (independant variables) and the target variable (dependant variable)
#sourcevars = d[["agr","art"] #all columns except the last one
#targetvar = df[1] #only the last column

sourcevars = df1[df1.columns[-31:-1]] # Remember, Python is 0-offset! The "3rd" entry is at slot 2.           
targetvar = df1[df1.columns[1:18]] # Remember, Python is 0-offset! The "3rd" entry is at slot 2.
               
X = sourcevars
y = targetvar 

#targetvar.head()
#print(y)
#print(X)

In [60]:
# b) Construct Datasets for Training and Evaluation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.25, random_state=42)
print("Number samples in training: ", len(X_train))
print("Number samples in testing: ", len(X_test))


Number samples in training:  514
Number samples in testing:  172


In [0]:
#!pip install -q skmultilearn.adapt
!pip install scikit-multilearn


     |████████████████████████████████| 92kB 3.4MB/s 


In [64]:
#a) Decision Tree Classifier
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn import metrics
#from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
tree_giniIndex = ClassifierChain(DecisionTreeClassifier(criterion='entropy', max_depth=17, class_weight="balanced"))

# train
tree_giniIndex.fit(X_train, y_train)

# predict
y_predict_DT = tree_giniIndex.predict(X_test)

#accuracy_DT = accuracy_score(y_test,y_predict_DT)
#print(accuracy_DT)

f1_score_DT = f1_score(y_test,y_predict_DT, average=None)
#f1_score_DT = f1_score(y_test,y_predict_DT, average="micro")
#f1_score_DT = f1_score(y_test,y_predict_DT, average="macro")
#f1_score_DT = f1_score(y_test,y_predict_DT, average="weighted")
#f1_score_DT = f1_score(y_test,y_predict_DT, average="binary")

print(f1_score_DT)



[0.         0.         0.34042553 0.28571429 0.5        0.35294118
 0.35616438 0.26666667 0.         0.         0.25806452 0.24137931
 0.25       0.13793103 0.09756098 0.14285714 0.15384615]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [62]:
#b) KneighborsClassifier
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
Knclassifier = ClassifierChain(KNeighborsClassifier())

# train
Knclassifier.fit(X_train, y_train)

# predict
y_predict_KN = Knclassifier.predict(X_test)

accuracy_KN= accuracy_score(y_test,y_predict_KN)

print(accuracy_KN)

0.11627906976744186


In [0]:
from sklearn.tree import DecisionTreeClassifier
tree_giniIndex = DecisionTreeClassifier(criterion='entropy', max_depth=6, class_weight="balanced")
tree_giniIndex.fit(X_train, y_train)

In [63]:
#EXPERIMENT 1 (EDIT DATASET AND TO MULTILABEL)

# a) Train and evaluate a decision treel model

# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn import metrics
from sklearn.metrics import f1_score

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
tree_giniIndex = ClassifierChain(DecisionTreeClassifier(criterion='entropy', max_depth=6, class_weight="balanced"))

# train
tree_giniIndex.fit(X_train, y_train)



# Perform 10-fold cross-validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, shuffle=True, random_state=2)
tree_scores = cross_val_score(tree_giniIndex, X, y, cv=kfold, scoring="f1_weighted")
print("Cross-validation score:\n{}".format(tree_scores))
print("Average cross-validation score: {:.2f}".format(tree_scores.mean()))
print("Standard deviation: {:.2f}".format(tree_scores.std()))

# Tune Decision Tree hyperparameter using cross-validation
best_score = 0
for max_d in [1,3,5,7,9,11,13,15,17]:
    
    tree_giniIndex = DecisionTreeClassifier(max_depth=max_d).fit(X_train, y_train)
    fold_accuracies = cross_val_score(tree_giniIndex, X_train, y_train, cv=kfold, scoring="f1_weighted") 
    score = fold_accuracies.mean()
    
    print("Score for depth of " + str(max_d) + "on validation set is {:0.2f}".format(score))
    
    #Best score for best optimal model
    
    if score > best_score:
        best_param = {'max_depth': max_d}
        best_score = score
        
#Part of the answer of question 3        
tree_giniIndex = DecisionTreeClassifier(**best_param)
tree_giniIndex.fit(X_train, y_train)
test_score = tree_giniIndex.score(X_test, y_test)
print("Best score on cross-validation: {:0.2f}".format(best_score))
print("Best parameters: {}".format(best_param))
print("Test set score: {:.2f}".format(test_score))


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning:

Cross-validation score:
[0.19621352 0.19413478 0.20927832 0.1910213  0.13248384 0.14054292
 0.08928964 0.16205062 0.14487933 0.16763895]
Average cross-validation score: 0.16
Standard deviation: 0.03
Score for depth of 1on validation set is 0.00
Score for depth of 3on validation set is 0.00


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Score for depth of 5on validation set is 0.02
Score for depth of 7on validation set is 0.09


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Score for depth of 9on validation set is 0.13
Score for depth of 11on validation set is 0.19


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

Score for depth of 13on validation set is 0.21
Score for depth of 15on validation set is 0.22
Score for depth of 17on validation set is 0.23
Best score on cross-validation: 0.23
Best parameters: {'max_depth': 17}
Test set score: 0.20


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning:

In [0]:
from sklearn import metrics
from sklearn.metrics import SCORERS


sorted(sklearn.metrics.SCORERS.keys())

NameError: ignored

In [0]:
#EXPERIMENT 1 (EDIT DATASET AND TO MULTILABEL)


# b) Nearest Neighbors (K-NN)
# Normalize data
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

# Tune hyperparameters using cross-validation for K-NN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=2)

best_score = 0

for p in [1, 2, 3, 4, 5]:
    for n in [1, 2, 3, 4, 5, 6]:   
        clf = KNeighborsClassifier(n_neighbors=n, p=p, metric='minkowski')
        clf.fit(X_train_norm, y_train)
            
        fold_accuracies = cross_val_score(clf, X_train, y_train, cv=kfold, scoring="accuracy")  
        score = fold_accuracies.mean()
        print("Score for p " + str(p) + ", score of neighbors " + str(n) + " on validation set is {:0.2f}".format(score))
    
        if score > best_score:
            best_param = {"p": p, "neighbors": n}
            best_score = score

#rebuild the model and evaluate the test score                
clf = KNeighborsClassifier(**best_param)
clf.fit(X_train_norm, y_train)
test_score = clf.score(X_test_norm, y_test)
print("Best score on cross-validation: {:0.2f}".format(best_score))
print("Best parameters: {}".format(best_param))
print("Test set score: {:.2f}".format(test_score))


In [0]:
#3 Optimal Decision Tree model predictive performance
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

tree_giniIndex = DecisionTreeClassifier(criterion='entropy', max_depth=3)
tree_giniIndex.fit(X_train, y_train)

y_predictedDT = tree_giniIndex.predict(X_test_scaled)
print(metrics.classification_report(y_predictedDT, y_test))
print(metrics.accuracy_score(y_predictedDT, y_test))

#3 Optimal SVM model predictive performance
from sklearn import metrics
from sklearn.svm import SVC

poly_kernel_svm_clf = SVC(kernel="poly", C = 1, degree=0, gamma=0.1)
poly_kernel_svm_clf.fit(X_train_scaled, y_train)
y_predictedSVM = poly_kernel_svm_clf.predict(X_test_scaled)
print(metrics.classification_report(y_predictedSVM, y_test))
print(metrics.accuracy_score(y_predictedSVM, y_test))


In [0]:
# Visualize confusion matrix for the Naive Bayes
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

mat = confusion_matrix(y_predictedNB, y_test)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')

# Visualize confusion matrix for Decision Tree
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

mat = confusion_matrix(y_predictedDT, y_test)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')

# Visualize confusion matrix for the SVM
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 

mat = confusion_matrix(y_predictedSVM, y_test)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')


In [0]:
#MULTILABEL CONFUSION MATRIX
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_true, y_pred, sample_weight=None, labels=None, samplewise=False)



In [0]:
#EXPERIMENT 2
# Plot learning curves
def plot_learning_curves(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    train_errors, test_errors = [], []
   # change de len for m in range(1, len(x_train)):
    for m in range(1, 5):
        model.fit(x_train[:m], y_train[:m])
        y_train_predict = model.predict(x_train[:m])
        y_test_predict = model.predict(x_test)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        test_errors.append(mean_squared_error(y_test_predict, y_test))
        plt.plot(train_errors, "r-+", linewidth = 2, label = "Training Data")
        plt.plot(test_errors, "b-+", linewidth = 3, label = "Test Data")
        plt.ylabel("Mean squared error")
        plt.title("Learning Curves")
        plt.legend()
        
linear_reg_model = linear_model.LinearRegression()
plot_learning_curves(linear_reg_model, x_real, y_real)

NameError: ignored

In [0]:
tree_giniIndex
from sklearn import linear_model
from sklearn.metrics import coverage_error
import matplotlib.pyplot as plt

def plot_learning_curves(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    train_errors, test_errors = [], []
    for m in range(1, 1280, 30):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_test_predict = model.predict(X_test)
        train_errors.append(mean_squared_error(y_train_predict, y_train[:m]))
        test_errors.append(mean_squared_error(y_test_predict, y_test))
        plt.plot(train_errors, "r-", linewidth = 1)
        plt.plot(test_errors, "b-", linewidth = 1)
        plt.ylabel('Mean squared error')
        plt.title('Linear_Learning Curves')
        plt.ylim(0, 5)

linear_reg_model = linear_model.LinearRegression()
plot_learning_curves(linear_reg_model, X, y_withNoise)

In [0]:
#Using skmultilearn

from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=20)

# train
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

accuracy_score(y_test,predictions)